# Masakhane - Reverse Machine Translation for African Languages (Using JoeyNMT)

> ## NB
>### - The purpose of this Notebook is to build models that translate African languages(target language) *into* English(source language). This will allow us to in future be able to make translations from one African language to the other. If you'd like to translate *from* English, please use [this](https://github.com/masakhane-io/masakhane-mt/blob/master/starter_notebook.ipynb) starter notebook instead.

>### - We call this reverse training because normally we build models that make translations from the source language(English) to the target language. But in this case we are doing the reverse; building models that make translations from the target language to the source(English)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

# Notes:

This is a notebook for training a nr - en translation model using the JW300 dataset.

We train on two configurations, one 'small' one, which uses the default config, and a 'large' one, which uses the suggestions and increases some of the parameters. Both of these models were trained for 100 epochs.

The larger model gave the best results, specifically:
```
2021-10-10 06:05:03,867 - INFO - joeynmt.prediction -  dev bleu[13a]:  36.14 [Beam search decoding with beam size = 5 and alpha = 1.0]
2021-10-10 06:08:01,274 - INFO - joeynmt.prediction - test bleu[13a]:  40.56 [Beam search decoding with beam size = 5 and alpha = 1.0]
 ```

## Nonstandard Changes

While most of this notebook is the same as the original template notebook, we made a few differences, mainly with regards to the joeynmt installation, as we has some problems with that.

Instead of `git clone ... && cd ... && pip install .`, we do the following:

```
! pip install joeynmt
! pip install torch==1.9.0
! pip install --upgrade sacrebleu==1.5.1


# make the directories since we didn't clone
!mkdir -p joeynmt/configs
!mkdir -p joeynmt/data
!mkdir -p joeynmt/scripts

# get the build_vocab.py script

!cd joeynmt/scripts && wget https://raw.githubusercontent.com/joeynmt/joeynmt/master/scripts/build_vocab.py
```

------


We also do not use google colab, and as such the google drive specific code (e.g. mounting the drive) is commented out. This can be uncommented as needed



Since we trained two models, and the second one was the best performing one, when using this notebook, feel free to comment out the first training cycle, as it makes the whole process take longer. We just leave it here as a log though.



A final note, we did not run the training commands in the notebook, but rather in a terminal, as that was more stable and disconnections were less brutal. The output will be shown here, and it should be completely reproducible by running the commands in the notebook instead.

In [54]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "nr" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "nr_to_english" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
!mkdir -p "nlp_things/masakhane/$tgt-$src-$tag"
os.environ["gdrive_path"] = "nlp_things/masakhane/%s-%s-%s" % (target_language, source_language, tag)

In [21]:
# Install opus-tools
! pip install opustools-pkg

In [22]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz

# Only 9MBs! Very low resourced.


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-nr.xml.gz not found. The following files are available for downloading:

   1 MB https://object.pouta.csc.fi/OPUS-JW300/v1c/xml/en-nr.xml.gz
 274 MB https://object.pouta.csc.fi/OPUS-JW300/v1c/xml/en.zip
   9 MB https://object.pouta.csc.fi/OPUS-JW300/v1c/xml/nr.zip

 284 MB Total size
./JW300_latest_xml_en-nr.xml.gz ... 100% of 1 MB
./JW300_latest_xml_en.zip ... 100% of 274 MB
./JW300_latest_xml_nr.zip ... 100% of 9 MB


In [23]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en

# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2021-10-08 19:30:06--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.05s   

2021-10-08 19:30:07 (5.48 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2021-10-08 19:30:07--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-nr.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 2

In [24]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
    for line in f:
        en_test_sents.add(line.strip())
        j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [25]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 3300/105999 lines since contained in test set.


,source_sentence,target_sentence
0,"July 1 , 2010","Arhostosi 1 , 2010"
1,Do You Know God by Name ?,Umazi Kuhle na UZimu ?
2,FROM OUR COVER,EZIKHAMBISANA NESIHLOKO ESINGAPHANDLE


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [26]:
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/home-mscluster/mbeukman/anaconda3/envs/nlp_q2/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [27]:
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np
from os import cpu_count
from functools import partial
from multiprocessing import Pool


# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
    candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
    if len(candidates) > 0:
        return process.extractOne(sample, candidates)[1]
    else:
        return np.nan

In [28]:
# start_time = time.time()
# ### iterating over pandas dataframe rows is not recomended, let use multi processing to apply the function

# with Pool(cpu_count()-1) as pool:
#     scores = pool.map(partial(fuzzfilter, candidates=list(en_test_sents), pad=5), df_pp['source_sentence'])
# hours, rem = divmod(time.time() - start_time, 3600)
# minutes, seconds = divmod(rem, 60)
# print("done in {}h:{}min:{}seconds".format(hours, minutes, seconds))

# # Filter out "almost overlapping samples"
# df_pp = df_pp.assign(scores=scores)
# df_pp = df_pp[df_pp['scores'] < 95]

In [29]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
    for index, row in stripped.iterrows():
        src_file.write(row["source_sentence"]+"\n")
        trg_file.write(row["target_sentence"]+"\n")

with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
    for index, row in dev.iterrows():
        src_file.write(row["source_sentence"]+"\n")
        trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.en <==
8 David loved music and he was skilled at it .
We invite you to take a careful look at them .
Jenney , a mother in New Zealand , says , “ For a few years after my son was diagnosed with spina bifida , I would be exhausted and weepy if I tried to do anything extra around the home . ”
In the region where I lived , poverty was widespread , and the work it took to make ends meet was exhausting .
Clearly , Jehovah values such qualities more than any material treasures .
His pain did not disappear overnight , but it did not consume him either .
It was about an earthquake in Sumatra that in 2004 triggered the most devastating tsunamis in recorded history .
As anointed Christians , the members of that slave “ keep following the Lamb no matter where he goes . ”
What example does Jehovah provide regarding self-control ?
( Luke 23 : 18-23 ) Yes , they contributed to Jesus ’ being killed .

==> train.nr <==
8 UDavidi bekathanda umbhino begodu gade anekghono lawo .
Njeke sibawa naw



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [30]:
# Install JoeyNMT
# ! pip install joeynmt
# ! git clone https://github.com/joeynmt/joeynmt.git
# ! cd joeynmt; pip3 install .
# Install Pytorch with GPU support v1.7.1.
# ! pip install torch==1.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
# ! pip install torch==1.9.0 -f https://download.pytorch.org/whl/torch_stable.html
# !pip uninstall -y torchtext && pip install torchtext

In [ ]:
# I had trouble with joey git installation, so I just installed it through here, and made the requisite directories.

# Install using pip
! pip install joeynmt
! pip install torch==1.9.0
! pip install --upgrade sacrebleu==1.5.1


!mkdir -p joeynmt/configs
!mkdir -p joeynmt/data
!mkdir -p joeynmt/scripts

!cd joeynmt/scripts && wget https://raw.githubusercontent.com/joeynmt/joeynmt/master/scripts/build_vocab.py

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [31]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data",target_language + source_language ) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$tgt$src/train.bpe.$src joeynmt/data/$tgt$src/train.bpe.$tgt --output_path joeynmt/data/$tgt$src/vocab.txt

# Some output
! echo "BPE NR Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$tgt$src/vocab.txt  # Herman

bpe.codes.4000	dev.en	     test.bpe.nr     test.nr	   train.en
dev.bpe.en	dev.nr	     test.en	     train.bpe.en  train.nr
dev.bpe.nr	test.bpe.en  test.en-any.en  train.bpe.nr  vocab.txt
bpe.codes.4000	dev.en	     test.bpe.nr     test.nr	   train.en
dev.bpe.en	dev.nr	     test.en	     train.bpe.en  train.nr
dev.bpe.nr	test.bpe.en  test.en-any.en  train.bpe.nr
BPE Afrikaans Sentences
Lokho kwenza bona ng@@ az@@ iwe njengom@@ untu ong@@ akath@@ emb@@ eki .
Kwathi bona ng@@ ifund@@ e iqiniso , akhenge ngis@@ av@@ uma uku@@ r@@ aga n@@ ent@@ wel@@ ey@@ o , nanyana umb@@ ereg@@ o lo bew@@ ub@@ had@@ ela kangaka .
Ng@@ isibonelo esihle eb@@ as@@ any@@ an@@ eni bami abab@@ ili , begodu ng@@ ikgh@@ ona noku@@ b@@ ereg@@ iswa ebandleni .
E@@ ban@@ twini abab@@ had@@ el@@ isa umth@@ elo nak@@ il@@ abo eng@@ ib@@ ereg@@ isana nabo ng@@ az@@ iwa njengom@@ untu oth@@ embekileko . ”
UR@@ u@@ te waf@@ ud@@ ukela kw@@ a - Israyeli lapho ebek@@ azoku@@ k@@ gh@@ ona uku@@ lotjha khona uZimu weqiniso .
Co

In [33]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

bpe.codes.4000	dev.en	     test.bpe.nr     test.nr	   train.en
dev.bpe.en	dev.nr	     test.en	     train.bpe.en  train.nr
dev.bpe.nr	test.bpe.en  test.en-any.en  train.bpe.nr


# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [42]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (target_language, source_language)
# gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{target_language}{source_language}_reverse_transformer"

data:
    src: "{target_language}"
    trg: "{source_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "none"
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "noam"              # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 8192
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 100                  # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_reverse_transformer_nr_to_en-first-try"
    overwrite: True              # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_reverse_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above


**Note, we ran this command in a normal shell, and the output can be found in the next cell**

In [ ]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_reverse_$tgt$src.yaml

In [1]:
!cat logs_run_nr_to_en_100_epochs

2021-10-08 19:52:07,219 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-10-08 19:52:07,283 - INFO - joeynmt.data - Loading training data...
2021-10-08 19:52:08,768 - INFO - joeynmt.data - Building vocabulary...
2021-10-08 19:52:08,993 - INFO - joeynmt.data - Loading dev data...
2021-10-08 19:52:09,010 - INFO - joeynmt.data - Loading test data...
2021-10-08 19:52:09,045 - INFO - joeynmt.data - Data loaded.
2021-10-08 19:52:09,046 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-10-08 19:52:09,283 - INFO - joeynmt.model - Enc-dec model built.
2021-10-08 19:52:09,295 - INFO - joeynmt.training - Total params: 12141056
2021-10-08 19:52:11,666 - INFO - joeynmt.helpers - cfg.name                           : nren_reverse_transformer
2021-10-08 19:52:11,666 - INFO - joeynmt.helpers - cfg.data.src                       : nr
2021-10-08 19:52:11,666 - INFO - joeynmt.helpers - cfg.data.trg                       : en
2021-10-08 19:52:11,667 - INFO - joeynmt.helpers - cf

In [ ]:
os.environ['FILE'] = f"{os.environ['gdrive_path']}/models/{os.environ['src']}{os.environ['tgt']}_reverse_transformer-models/"

!echo $FILE

In [47]:
# Copy the created models from the notebook storage to google drive for persistant storage
!mkdir -p "$FILE"

!cp -r joeynmt/models/${tgt}${src}_reverse_transformer_nr_to_en-first-try/* "$FILE"



In [48]:
# Output our validation accuracy
! cat "$FILE/validations.txt"

Steps: 500	Loss: 62255.28516	PPL: 9.90640	bleu: 0.36239	LR: 0.00049411	*
Steps: 1000	Loss: 50472.49609	PPL: 6.41835	bleu: 2.63148	LR: 0.00098821	*
Steps: 1500	Loss: 42351.22656	PPL: 4.75888	bleu: 7.84905	LR: 0.00080687	*
Steps: 2000	Loss: 38393.57812	PPL: 4.11333	bleu: 12.70668	LR: 0.00069877	*
Steps: 2500	Loss: 36433.80078	PPL: 3.82686	bleu: 13.58898	LR: 0.00062500	*
Steps: 3000	Loss: 34767.03516	PPL: 3.59897	bleu: 16.13477	LR: 0.00057054	*
Steps: 3500	Loss: 33388.32031	PPL: 3.42076	bleu: 17.23224	LR: 0.00052822	*
Steps: 4000	Loss: 32373.90039	PPL: 3.29530	bleu: 17.86603	LR: 0.00049411	*
Steps: 4500	Loss: 31434.34180	PPL: 3.18320	bleu: 20.12044	LR: 0.00046585	*
Steps: 5000	Loss: 30865.75000	PPL: 3.11723	bleu: 20.36163	LR: 0.00044194	*
Steps: 5500	Loss: 30126.25781	PPL: 3.03346	bleu: 21.67347	LR: 0.00042137	*
Steps: 6000	Loss: 29731.27148	PPL: 2.98965	bleu: 21.72692	LR: 0.00040344	*
Steps: 6500	Loss: 29140.94922	PPL: 2.92534	bleu: 22.19796	LR: 0.00038761	*
Steps: 7000	Loss: 28676.16406

In [49]:
# Test our model
! cd joeynmt; python3 -m joeynmt test ../"$FILE/config.yaml"

2021-10-09 06:12:59,357 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-10-09 06:12:59,360 - INFO - joeynmt.data - Building vocabulary...
2021-10-09 06:12:59,632 - INFO - joeynmt.data - Loading dev data...
2021-10-09 06:12:59,648 - INFO - joeynmt.data - Loading test data...
2021-10-09 06:12:59,681 - INFO - joeynmt.data - Data loaded.
2021-10-09 06:12:59,745 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 2, batch_size per device: 9000 (with beam_size)
2021-10-09 06:13:02,909 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-10-09 06:13:03,200 - INFO - joeynmt.model - Enc-dec model built.
2021-10-09 06:13:03,303 - INFO - joeynmt.prediction - Decoding on dev set (data/nren/dev.bpe.en)...
2021-10-09 06:14:23,920 - WARNING - sacrebleu - That's 100 lines that end in a tokenized period ('.')
2021-10-09 06:14:23,921 - WARNING - sacrebleu - It looks like you forgot to detokenize your test data, which may hurt your score.
2021-10-09 06:14:23,921 - WARNING

# Try and experiment with the parameters => Make the model bigger


Here we implement some of the suggestions for the configuration, and generally increase the size of the parameters.

In [50]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (target_language, source_language)
# gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{target_language}{source_language}_reverse_transformer"

data:
    src: "{target_language}"
    trg: "{source_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "noam"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 100                  # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_reverse_transformer_bigger_model_1009"
    overwrite: True              # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 8             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 512   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 512         # TODO: Increase to 512 for larger data.
        ff_size: 2048            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 8              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 512    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 512         # TODO: Increase to 512 for larger data.
        ff_size: 2048            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_reverse_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

In [ ]:
!cd joeynmt; python3 -m joeynmt train configs/transformer_reverse_$tgt$src.yaml

In [16]:
!cat logs_run_nr_to_en_100_epochs_larger_model_1009_1633753204

2021-10-09 06:20:06,815 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-10-09 06:20:06,877 - INFO - joeynmt.data - Loading training data...
2021-10-09 06:20:08,298 - INFO - joeynmt.data - Building vocabulary...
2021-10-09 06:20:08,517 - INFO - joeynmt.data - Loading dev data...
2021-10-09 06:20:08,529 - INFO - joeynmt.data - Loading test data...
2021-10-09 06:20:08,559 - INFO - joeynmt.data - Data loaded.
2021-10-09 06:20:08,559 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-10-09 06:20:09,331 - INFO - joeynmt.model - Enc-dec model built.
2021-10-09 06:20:09,342 - INFO - joeynmt.training - Total params: 46302208
2021-10-09 06:20:11,795 - INFO - joeynmt.helpers - cfg.name                           : nren_reverse_transformer
2021-10-09 06:20:11,795 - INFO - joeynmt.helpers - cfg.data.src                       : nr
2021-10-09 06:20:11,795 - INFO - joeynmt.helpers - cfg.data.trg                       : en
2021-10-09 06:20:11,795 - INFO - joeynmt.helpers - cf

In [11]:
os.environ['FILE'] = f"nlp_things/masakhane/models/{os.environ['tgt']}{os.environ['src']}-larger-model-final/"

!echo $FILE

nlp_things/masakhane/models/nren-larger-model-final/


In [ ]:
# Copy the created models from the notebook storage to google drive for persistant storage
!mkdir -p "$FILE"

!cp -r joeynmt/models/"$tgt$src"_reverse_transformer_bigger_model_1009/* "$FILE"



In [12]:
# Output our validation accuracy
! cat "$FILE/validations.txt"

Steps: 500	Loss: 61180.94531	PPL: 9.52203	bleu: 0.73583	LR: 0.00034939	*
Steps: 1000	Loss: 50408.64062	PPL: 6.40327	bleu: 2.55773	LR: 0.00069877	*
Steps: 1500	Loss: 43601.39062	PPL: 4.98315	bleu: 5.60927	LR: 0.00057054	*
Steps: 2000	Loss: 39644.05469	PPL: 4.30722	bleu: 9.94033	LR: 0.00049411	*
Steps: 2500	Loss: 37151.26953	PPL: 3.92934	bleu: 12.03066	LR: 0.00044194	*
Steps: 3000	Loss: 35565.21484	PPL: 3.70636	bleu: 14.14590	LR: 0.00040344	*
Steps: 3500	Loss: 34094.69141	PPL: 3.51093	bleu: 15.89781	LR: 0.00037351	*
Steps: 4000	Loss: 32934.26172	PPL: 3.36402	bleu: 15.88460	LR: 0.00034939	*
Steps: 4500	Loss: 32159.54883	PPL: 3.26938	bleu: 17.79568	LR: 0.00032940	*
Steps: 5000	Loss: 31214.65820	PPL: 3.15755	bleu: 18.67098	LR: 0.00031250	*
Steps: 5500	Loss: 30482.10742	PPL: 3.07349	bleu: 18.58273	LR: 0.00029796	*
Steps: 6000	Loss: 29903.44727	PPL: 3.00867	bleu: 19.84278	LR: 0.00028527	*
Steps: 6500	Loss: 29464.80469	PPL: 2.96045	bleu: 20.36883	LR: 0.00027408	*
Steps: 7000	Loss: 28848.15234	

In [ ]:
# Test our model
! cd joeynmt; python3 -m joeynmt test ../"$FILE/config.yaml"

In [13]:
!cat log_run_nr_to_en_test

2021-10-10 06:02:52,312 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-10-10 06:02:52,317 - INFO - joeynmt.data - Building vocabulary...
2021-10-10 06:02:52,593 - INFO - joeynmt.data - Loading dev data...
2021-10-10 06:02:52,608 - INFO - joeynmt.data - Loading test data...
2021-10-10 06:02:52,641 - INFO - joeynmt.data - Data loaded.
2021-10-10 06:02:52,702 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 2, batch_size per device: 9000 (with beam_size)
2021-10-10 06:02:55,520 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-10-10 06:02:56,272 - INFO - joeynmt.model - Enc-dec model built.
2021-10-10 06:02:56,461 - INFO - joeynmt.prediction - Decoding on dev set (data/nren/dev.bpe.en)...
2021-10-10 06:05:03,492 - WARNING - sacrebleu - That's 100 lines that end in a tokenized period ('.')
2021-10-10 06:05:03,493 - WARNING - sacrebleu - It looks like you forgot to detokenize your test data, which may hurt your score.
2021-10-10 06:05:03,493 - WARNING